In [2]:
import pandas as pd
import torch
import matplotlib as mpl
import matplotlib.pyplot as plt
from datasets import DatasetDict, load_from_disk, Dataset, concatenate_datasets
import re

/opt/conda/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import json
with open('/opt/ml/input/data/KorQuAD_2.1/train/korquad2.1_train_00.json','r') as f:
    train_data = json.load(f)

In [4]:
train_data.keys()

dict_keys(['version', 'data'])

In [5]:
type(train_data)

dict

In [6]:
train_data['version']

'KorQuAD_2.0_train'

In [7]:
len(train_data['data'])

1000

In [8]:
train_data['data'][0].keys()

dict_keys(['title', 'url', 'context', 'raw_html', 'qas'])

In [9]:
train_data['data'][999].keys()

dict_keys(['title', 'url', 'context', 'raw_html', 'qas'])

In [65]:
# http://zeany.net/46
def preprocess_context(example):
    body = re.sub('<table.*?>.*?</table>', '', example, 0, re.I|re.S)
    body = re.sub('<script.*?>.*?</script>', '', body, 0, re.I|re.S)
    text = re.sub('<.+?>', '', body, 0, re.I|re.S)
    space = re.sub('&nbsp;| |\t|\r|\n|\xa0', ' ', text)
    one_space = re.sub(' +', ' ', space)
    return one_space

In [70]:
import json
whole_list = []
train_dict = {}
answer_dict = {}
c = 0
for j in range(0,39):
    file_j = ('0'+str(j))[-2:]
    with open('/opt/ml/input/data/KorQuAD_2.1/train/korquad2.1_train_{}.json'.format(file_j),'r') as f:
        train_data = json.load(f)
        for i in range(len(train_data['data'])):
            answers = train_data['data'][i]['qas'][0]
            answer_dict['answer_start'] = [len(preprocess_context(train_data['data'][i]['context'][:answers['answer']['answer_start']]).lstrip())]
            answer_dict['text'] = [preprocess_context(answers['answer']['text']).strip()]
            count_id = ('000000' + str(c))[-6:]
            train_dict['title']=train_data['data'][i]['title']
            train_dict['context']= preprocess_context(train_data['data'][i]['context']).strip()
            train_dict['question']=answers['question']
            train_dict['id']='korquad-{}-{}'.format(file_j,count_id)
            train_dict['answers']=answer_dict
            train_dict['document_id']=int(answers['id'])
            train_dict['__index_level_0__']=0
            if len(answer_dict['text'][0])>0:
                if len(train_dict['context']) > answer_dict['answer_start'][0]:
                    if train_dict['context'][answer_dict['answer_start'][0]] != answer_dict['text'][0][0]:
                        continue
                    else:
                        whole_list.append(train_dict)
                else:
                    continue
            else:
                continue
            answer_dict={}
            train_dict={}
            c = c + 1

In [71]:
len(whole_list)

31660

In [79]:
for idx,i in enumerate(range(0,35000,2500)):
    with open('/opt/ml/input/data/KorQuAD_2.1/train/no_table/train_notab_{}.json'.format(idx),'w') as outfile:
        json.dump(whole_list[i:i+2500],outfile)
        print(i,i+2500)

0 2500
2500 5000
5000 7500
7500 10000
10000 12500
12500 15000
15000 17500
17500 20000
20000 22500
22500 25000
25000 27500
27500 30000
30000 32500
32500 35000


In [89]:
import json
dev_list = []
train_dict = {}
answer_dict = {}
c = 0
for j in range(0,5):
    file_j = ('0'+str(j))[-2:]
    with open('/opt/ml/input/data/KorQuAD_2.1/dev/korquad2.1_dev_{}.json'.format(file_j),'r') as f:
        train_data = json.load(f)
        for i in range(len(train_data['data'])):
            answers = train_data['data'][i]['qas'][0]
            answer_dict['answer_start'] = [len(preprocess_context(train_data['data'][i]['context'][:answers['answer']['answer_start']]).lstrip())]
            answer_dict['text'] = [preprocess_context(answers['answer']['text']).strip()]
            count_id = ('000000' + str(c))[-6:]
            train_dict['title']=train_data['data'][i]['title']
            train_dict['context']= preprocess_context(train_data['data'][i]['context']).strip()
            train_dict['question']=answers['question']
            train_dict['id']='korquad-{}-{}'.format(file_j,count_id)
            train_dict['answers']=answer_dict
            train_dict['document_id']=int(answers['id'])
            train_dict['__index_level_0__']=0
            if len(answer_dict['text'][0])>0:
                if len(train_dict['context']) > answer_dict['answer_start'][0]:
                    if train_dict['context'][answer_dict['answer_start'][0]] != answer_dict['text'][0][0]:
                        continue
                    else:
                        dev_list.append(train_dict)
                else:
                    continue
            else:
                continue
            answer_dict={}
            train_dict={}
            c = c + 1

In [90]:
len(dev_list)

3864

In [91]:
for idx,i in enumerate(range(0,4000,500)):
    with open('/opt/ml/input/data/KorQuAD_2.1/dev/no_table/valid_notab_{}.json'.format(idx),'w') as outfile:
        json.dump(dev_list[i:i+500],outfile)
        print(i,i+500)

0 500
500 1000
1000 1500
1500 2000
2000 2500
2500 3000
3000 3500
3500 4000


In [47]:
ds = Dataset.from_json('/opt/ml/input/data/KorQuAD_2.1/train/train.json')

Using custom data configuration default-5f30fd53ca4be2a5
Found cached dataset json (/opt/ml/.cache/huggingface/datasets/json/default-5f30fd53ca4be2a5/0.0.0)


In [48]:
dv = Dataset.from_json('/opt/ml/input/data/KorQuAD_2.1/dev/valid.json')

Using custom data configuration default-56fe0ff52f29f51d
Found cached dataset json (/opt/ml/.cache/huggingface/datasets/json/default-56fe0ff52f29f51d/0.0.0)


In [98]:
ds['context'][0][ds['answers'][0]['answer_start'][0]]

'나'

In [99]:
ds['answers'][0]['text'][0][0]

'나'

In [100]:
train_dataset = load_from_disk('/opt/ml/input/data/train_dataset')

In [101]:
train_dataset

DatasetDict({
    train: Dataset({
        features: ['title', 'context', 'question', 'id', 'answers', 'document_id', '__index_level_0__'],
        num_rows: 3952
    })
    validation: Dataset({
        features: ['title', 'context', 'question', 'id', 'answers', 'document_id', '__index_level_0__'],
        num_rows: 240
    })
})

In [102]:
concatenate_datasets([train_dataset['train'],ds])

Dataset({
    features: ['title', 'context', 'question', 'id', 'answers', 'document_id', '__index_level_0__'],
    num_rows: 42448
})

In [103]:
DatasetDict({'train' : ds, 'validation' : dv})

DatasetDict({
    train: Dataset({
        features: ['title', 'context', 'question', 'id', 'answers', 'document_id', '__index_level_0__'],
        num_rows: 38496
    })
    validation: Dataset({
        features: ['title', 'context', 'question', 'id', 'answers', 'document_id', '__index_level_0__'],
        num_rows: 4736
    })
})